In [ ]:
🪜 Step 1: Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

🎯 Step 2: Load and Inspect Dataset

In [ ]:
df = pd.read_csv('census_income.csv')

# Display info and preview
print(df.info())
df.head()


🔍 Step 3: Clean and Prepare Data

We’ll encode your target variable (income) and pick a few continuous predictors for visualization (like age, hours_per_week, education_num).

In [ ]:
# Clean target
df["income"] = df["income"].str.strip()
df["income"] = df["income"].replace({">50K": 1, "<=50K": 0})

# Drop missing values
df = df.dropna()

# Select features
features = ["age", "hours_per_week", "education_num"]
X = df[features].values
y = df["income"].values.reshape(-1, 1)


🎨 Step 4: Visualize Feature Relationships

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, feature in enumerate(features):
    sns.regplot(x=df[feature], y=df["income"], ax=axes[idx], scatter_kws={'alpha':0.3})
    axes[idx].set_title(f"{feature} vs Income (>50K)")
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel("Income (>50K)")
plt.tight_layout()
plt.show()


🧠 Step 5: Implement Linear Regression From Scratch

In [ ]:
class AdvancedLinearRegression:
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None
        self.residuals_ = None
    
    def fit(self, X, y):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        try:
            theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
            self.intercept_ = theta[0]
            self.coef_ = theta[1:]
            self.residuals_ = y - self.predict(X)
        except np.linalg.LinAlgError:
            raise ValueError("Matrix is singular. Check for multicollinearity.")
    
    def predict(self, X):
        return self.intercept_ + X @ self.coef_
    
    def r_squared(self, y_true, y_pred):
        ss_res = np.sum((y_true - y_pred) ** 2)
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
        return 1 - (ss_res / ss_tot)
    
    def summary(self, X, y):
        y_pred = self.predict(X)
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        n, p = X_b.shape
        mse = np.sum(self.residuals_ ** 2) / (n - p)
        cov_matrix = mse * np.linalg.inv(X_b.T @ X_b)
        se_coef = np.sqrt(np.diag(cov_matrix))
        
        t_stats = (np.concatenate([self.intercept_, self.coef_.flatten()]) / se_coef)
        p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n-p))
        
        print(f"Intercept: {self.intercept_[0]:.4f}")
        for i in range(len(self.coef_)):
            print(f"{features[i]} coef: {self.coef_[i][0]:.4f}")
        print(f"R²: {self.r_squared(y, y_pred):.4f}")


⚙️ Step 6: Train and Summarize

In [ ]:
model = AdvancedLinearRegression()
model.fit(X, y)
model.summary(X, y)


🔬 Step 7: Compare with scikit-learn

In [ ]:
sklearn_model = LinearRegression().fit(X, y)
print("\n--- Comparison with Scikit-learn ---")
print("Intercept:", sklearn_model.intercept_[0])
print("Coefficients:", sklearn_model.coef_[0])
print("R²:", sklearn_model.score(X, y))

📏 Step 8: Feature Scaling

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model_scaled = AdvancedLinearRegression()
model_scaled.fit(X_scaled, y)
print("\n--- After Feature Scaling ---")
model_scaled.summary(X_scaled, y)


🔁 Step 9: Cross Validation

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scores = []

for train_idx, test_idx in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    model_cv = AdvancedLinearRegression()
    model_cv.fit(X_train, y_train)
    y_pred = model_cv.predict(X_test)
    
    r2_scores.append(model_cv.r_squared(y_test, y_pred))

print(f"\nMean R² across 5 folds: {np.mean(r2_scores):.4f}")


📊 Step 10: Residual Analysis

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(model.predict(X), model.residuals_, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel("Predicted Income (>50K)")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.grid(True)
plt.show()
